# 多体系统数学规划教程
关于如何运行这些教程笔记本，请参见 [index](./index.ipynb)。


本教程展示了以下内容：
* 创建包含 IIWA 机械臂的 `MultibodyPlant`
* 通过编写自定义评估器，解决一个简单的逆运动学问题，该评估器可同时处理 `float` 和 `AutoDiffXd` 输入
* 在约束中使用自定义评估器
* 在代价函数中使用自定义评估器。

***待补充***：
* 使用 `pydrake.multibody.inverse_kinematics`。
* 使用 Meshcat 可视化。

### 重要说明

在深入编写用于 `MultibodyPlant` 的自定义评估器之前，请先查阅
[`pydrake.multibody.inverse_kinematics` 的 API 文档](https://drake.mit.edu/pydrake/pydrake.multibody.inverse_kinematics.html)。
你可能会发现你需要的功能已经被实现。

## 逆运动学问题

在本教程中，我们将解决一个简单的逆运动学问题：
使 Link 7 的原点与目标位置之间的距离达到指定值。  

我们将使用 `MathematicalProgram` 以两种不同方式求解该问题：首先
将评估器作为约束（设置最小和最大距离），
其次将评估器作为代价函数（尽可能接近目标）。
  
关于 `MathematicalProgram` 的更多信息，请参见
[`MathematicalProgram` 教程](./mathematical_program.ipynb)。

## 环境准备

首先，我们将导入必要的模块，并加载包含 IIWA 机械臂的 `MultibodyPlant`。

In [1]:
import numpy as np

from pydrake.math import RigidTransform
from pydrake.multibody.parsing import Parser
from pydrake.systems.analysis import Simulator
from pydrake.all import MultibodyPlant

from pydrake.solvers import MathematicalProgram, Solve

In [2]:
plant_f = MultibodyPlant(0.0)
iiwa_url = (
   "package://drake_models/iiwa_description/sdf/iiwa14_no_collision.sdf"
 )
(iiwa,) = Parser(plant_f).AddModels(url=iiwa_url)

# 定义一些帧的简写。
W = plant_f.world_frame()
L0 = plant_f.GetFrameByName("iiwa_link_0", iiwa)
L7 = plant_f.GetFrameByName("iiwa_link_7", iiwa)

plant_f.WeldFrames(W, L0)
plant_f.Finalize()

## 编写自定义评估器

我们的评估器通过自定义函数 `link_7_distance_to_target` 实现，因为 `inverse_kinematics` 子模块中尚未提供该功能。

注意：在 Python 中编写自定义评估器时，必须显式判断输入类型是 `float` 还是 `AutoDiffXd`，具体可见 `link_7_distance_to_target` 的实现。

In [3]:
# 分配 float 类型的 context，供评估器使用。
context_f = plant_f.CreateDefaultContext()
# 创建 AutoDiffXd 类型的 plant 及其 context。
plant_ad = plant_f.ToAutoDiffXd()
context_ad = plant_ad.CreateDefaultContext()

def resolve_frame(plant, F):
    """获取标量类型可能不同的 plant 中的帧。"""
    return plant.GetFrameByName(F.name(), F.model_instance())

# 定义目标位置。
p_WT = [0.1, 0.1, 0.6]

def link_7_distance_to_target(q):
    """计算 L7 原点与目标 T 之间的平方距离。"""
    # 根据数据类型选择 plant 和 context。
    if q.dtype == float:
        plant = plant_f
        context = context_f
    else:
        # 假定为 AutoDiff 类型。
        plant = plant_ad
        context = context_ad
    # 正向运动学。
    plant.SetPositions(context, iiwa, q)
    X_WL7 = plant.CalcRelativeTransform(
        context, resolve_frame(plant, W), resolve_frame(plant, L7))
    p_TL7 = X_WL7.translation() - p_WT
    return p_TL7.dot(p_TL7)

# 警告：如果你为约束返回标量，或为代价函数返回向量，
# 可能会遇到如下晦涩的报错：
# "Unable to cast Python instance to C++ type"
link_7_distance_to_target_vector = lambda q: [link_7_distance_to_target(q)]

## 优化问题的构建

### 方式一：在约束中使用自定义评估器

我们将用基本代价函数和自定义评估器作为约束来构建并求解该问题。

注意，这里使用了评估器的向量版本。

In [4]:
prog = MathematicalProgram()

q = prog.NewContinuousVariables(plant_f.num_positions())
# 定义标称构型。
q0 = np.zeros(plant_f.num_positions())

# 添加基本代价函数（将被解析为 QuadraticCost）。
prog.AddCost((q - q0).dot(q - q0))

# 添加基于自定义评估器的约束。
prog.AddConstraint(
    link_7_distance_to_target_vector,
    lb=[0.1], ub=[0.2], vars=q)

In [5]:
result = Solve(prog, initial_guess=q0)

print(f"是否成功? {result.is_success()}")
print(result.get_solution_result())
q_sol = result.GetSolution(q)
print(q_sol)

print(f"初始距离: {link_7_distance_to_target(q0):.3f}")
print(f"解的距离: {link_7_distance_to_target(q_sol):.3f}")

是否成功? True
SolutionResult.kSolutionFound
[ 0.18293302 -0.15345868  0.22224834 -1.24010409  0.01028776  0.28116727
  0.        ]
初始距离: 0.457
解的距离: 0.200


### 方式二：在代价函数中使用自定义评估器

我们将重新构建并求解该问题，这次将自定义评估器作为代价函数。

注意，这里使用了评估器的标量版本。

In [6]:
prog = MathematicalProgram()

q = prog.NewContinuousVariables(plant_f.num_positions())
# 定义标称构型。
q0 = np.zeros(plant_f.num_positions())

# 添加自定义代价函数。
prog.AddCost(link_7_distance_to_target, vars=q)

In [7]:
result = Solve(prog, initial_guess=q0)

print(f"是否成功? {result.is_success()}")
print(result.get_solution_result())
q_sol = result.GetSolution(q)
print(q_sol)

print(f"初始距离: {link_7_distance_to_target(q0):.3f}")
print(f"解的距离: {link_7_distance_to_target(q_sol):.3f}")

是否成功? True
SolutionResult.kSolutionFound
[ 2.94972565  0.97163332  0.47321684 -3.63325213  0.08684904  1.11349396
  0.        ]
初始距离: 0.457
解的距离: 0.000
